In [ ]:
import pysam
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../JKBio/')
import epigenetics.ChIP_helper as chiphelper
import Helper as helper
from pybedtools import BedTool
%load_ext autoreload
%autoreload 2

In [ ]:
project="enhancerCRISPrI"
version = "v3"

In [ ]:
mor3tf = pd.read_csv('../data/'+project+'/CRIPSR_guides/MV411_3ormore_TF_cobinding.csv',header=None,skiprows=1, names=['chrom','start','end'])
mor3tf['name'] = '3PTF'

In [ ]:
h3peaks = pd.read_csv('../data/'+project+'/CRIPSR_guides/MV411_H3k27ac_peaks.csv',header=None,skiprows=1,names=['chrom','start','end'])
h3peaks['name']='H3K27AC'

In [ ]:
merged = chiphelper.simpleMergedPeaks(pd.concat([mor3tf,h3peaks]))

In [ ]:
tads = pd.read_csv('../data/'+project+'/CRIPSR_guides/MV411_TAD_for_dcas9_target_TF.csv',skiprows=2, names=['name','chrom','start','end','size'])
tads = tads[tads.columns[[1,2,3,0]]]

In [ ]:
tads = pd.read_csv('../results/'+project+'/'+version+'_tadsv2.bed', sep='\t')

In [ ]:
peaks = pd.read_csv('..data/'+project+'/CRIPSR_guides/MV411_peaks.csv',header=None,skiprows=1,names=['chrom','start','end'])
peaks['name']='AllPeaks'

In [ ]:
merged[merged.columns[[0,1,2]]].to_csv('..data/'+project+'/'+version+'_merged.bed',sep='\t',index=False)
merged = BedTool('..data/'+project+'/'+version+'_merged.bed')

In [ ]:
merged.head()

In [ ]:
tads.to_csv('../results/'+project+'/'+version+'_tads.bed',sep='\t',index=False)
tads = BedTool('../results/'+project+'/'+version+'_tads.bed')

In [ ]:
tads.head()

In [ ]:
peaks.to_csv('../results/'+project+'/'+version+'_peaks.bed',sep='\t',index=False)
peaks = BedTool('../results/'+project+'/'+version+'_peaks.bed')

In [ ]:
positive = tads.intersect(merged)
positive.saveas('../results/'+project+'/'+version+'_positive.bed')

In [ ]:
positive.head()

In [ ]:
sizes = pd.read_csv('../data/genome_size.tsv',sep='\t',header=None,names=['chrom','size'])

In [ ]:
sizes['end']=sizes['size']
sizes['start'] = 0

In [ ]:
sizes['chrom'] = ['chr'+i for i in sizes['chrom']]

In [ ]:
sizes = sizes[sizes.columns[[0,-1,-2]]]

In [ ]:
sizes.to_csv('../results/'+project+'/'+version+'_sizes.bed',sep='\t',index=False)
siz = BedTool('sizes.bed')

In [ ]:
dfh3peaks = pd.read_csv('../data/'+project+'/CRIPSR_guides/MV411_H3k27ac_peaks.csv',header=None,skiprows=1,names=['chrom','start','end'])
dfh3peaks['name']='H3K27AC'

In [ ]:
dfpeaks = pd.read_csv('../data/'+project+'/CRIPSR_guides/MV411_peaks.csv',header=None,skiprows=1,names=['chrom','start','end'])
dfpeaks['name']='AllPeaks'

In [ ]:
omerged = chiphelper.simpleMergedPeaks(pd.concat([dfpeaks,dfh3peaks]))
omerged.chrom = [i[3:] for i in omerged.chrom]

In [ ]:
omerged['chrom'] = ['chr'+i for i in omerged['chrom']]

In [ ]:
omerged[omerged.columns[:3]].to_csv('../results/'+project+'/'+version+'_notneg.bed',sep='\t',index=False)
not_neg = BedTool('../results/'+project+'/'+version+'_notneg.bed')

In [ ]:
negative = siz.subtract(not_neg)

In [ ]:
negative=tads.intersect(negative)

In [ ]:
negative.head()

In [ ]:
negative.saveas('../results/'+project+'/'+version+'_negative.bed')